In [1]:
%%writefile data_processing.py
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from datasets import Dataset

classes=[0,1]
classes0 = ['Ei_tarkistettavissa_oleva_väite', 'Määrä', 'Ennuste', 'Korrelaatio/kaussatio', 'Lait/toimintasäännot', 'Muu', 'Henkilökohtainen_kokemus']

num_classes = len(classes)

def get_text_label(path):
    files=['Dana_annotation_1000_sentences_Henna.csv', 'DIME_claim_annotation_1000_sentences_Minttu.csv', 'DIME_claim_annotation_500_sentences_Pipsa.csv'] #, '3xNCS.json']
    
    fns = [os.path.join(path, f) for f in files]
    
    assert len(fns) == 3
    assert os.path.exists(fns[0])
    assert os.path.exists(fns[1])
    assert os.path.exists(fns[2])
    
    # Open and read the JSON file
    data1 = pd.read_csv(fns[0], sep=';')
    
    data2 = pd.read_csv(fns[1], sep=';')
    
    data3 = pd.read_csv(fns[2], sep=';')
    
    data3=data3[:501]
    
    datasets=[data1, data2, data3]
    texts=[]
    labels=[]
    
    for item in datasets:
        
        try:
            newtext=item['Lauseet']
        except:
            newtext=item['sentences']
        newlab=item['Kategoria']
        
        for it in newtext:
            if it == '':
                pass
            else:
                texts.append(it)
        for lab in newlab:
            if lab=='Ei_tarkistettavissa_oleva_väite':
                labels.append(0)    
            elif lab=='Henkilökohtainen_kokemus':
                labels.append(0)    
            elif lab=='Muu':
                labels.append(0)  
            elif lab=='':
                pass
            else:
                labels.append(1)
    
    print(f"labels[0] = {labels[0]}")
    print(f"texts[0] = {texts[0]}")
    return texts, labels

def get_data_splits(state_x, texts, labels):
    state_r=state_x

    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=state_r) #10 has been run
    X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=state_r)

    test_scores=y_test
    
    X_tr=[]
    y_train_n=[]
    i =0
    for item in X_train:
        new = str(item)
        X_tr.append(new)
        y_train_n.append(y_train[i])
        i+=1
    
    X_te=[]
    y_test_n=[]
    test_scores2=[]
    i =0
    for item in X_test:
        new = str(item)
        X_te.append(new)
        y_test_n.append(y_test[i])
        i+=1

    X_va=[]
    y_val_n=[]
    i =0
    for item in X_val:
        new = str(item)
        X_va.append(new)
        y_val_n.append(y_val[i])
        i+=1

    traind = pd.DataFrame()
    traind['Label'] = y_train_n
    traind['Text'] = X_tr
    train_df = traind[['Text', 'Label']]
    train_df.columns = ['text', 'label']

    testd = pd.DataFrame()
    testd['Label'] = y_test_n
    testd['Text'] = X_te
    test_df1 = testd[['Text', 'Label']]
    test_df1.columns = ['text', 'label']

    vald = pd.DataFrame()
    vald['Label'] = y_val_n
    vald['Text'] = X_va
    dev_df = vald[['Text', 'Label']]
    dev_df.columns = ['text', 'label']

    return train_df, test_df1, dev_df, test_scores2

def encode_labels(record):
    return {"label": [record[feature] for feature in classes]}


def shape_data(train_df, test_df1, dev_df, multi_label):
    train_ds = Dataset.from_dict(train_df)
    dev_ds = Dataset.from_dict(dev_df)
    test_ds1 = Dataset.from_dict(test_df1)
    
    if multi_label:
        train_ds = train_ds.map(encode_labels)
        test_ds1 = test_ds1.map(encode_labels)

    train_ds
    test_df = test_df1
    test_ds = test_ds1

    return train_ds, test_ds, dev_ds, test_df


Overwriting data_processing.py


In [48]:
## Inspect the data reading, spliting code
random_states=[1,2,3,4,5,6,7,8,9,10]

k=0
texts, labels = get_text_label(".")
for state_x in random_states:
    
    train_df, test_df1, dev_df, test_scores= get_data_splits(state_x, texts, labels)
    print("*"*10)
    display(train_df.head(1), test_df1.head(1), dev_df.head(1), test_scores)
    print("*"*10)

    train_ds, test_ds, dev_ds, test_df= shape_data(train_df, test_df1, dev_df, multi_label)

    break

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

,text,label
0,"Ylelle ei kerrota, onko viruksen taloon tuonut...",1


,text,label
0,"Jos päätetään hankkia omat kopterit, silloin k...",0


,text,label
0,– Verot eivät saa nousta.,0


[]

**********
<class 'pandas.core.frame.DataFrame'>
[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,

In [3]:
import pandas as pd


files=['Dana_annotation_1000_sentences_Henna.csv', 'DIME_claim_annotation_1000_sentences_Minttu.csv', 'DIME_claim_annotation_500_sentences_Pipsa.csv'] #, '3xNCS.json']

data1 = pd.read_csv(files[0], sep=';')

display(data1.head(5))

data1.head(5).Lauseet.values.tolist()


,id,Lauseet,Kategoria
0,3-12394959,Sitten yksi sotilaista kehotti toista “lopetta...,Ei_tarkistettavissa_oleva_väite
1,3-12394959,Amnestyn haastatteleman todistajan mukaan mies...,Muu
2,3-12394959,Naapurissa asuva henkilö on vahvistanut Amnest...,Muu
3,3-12394959,Idässä tullaan näkemään sodan suurimmat taistelut,Ennuste
4,3-12394959,Ruotsin maanpuolustuskorkeakoulun sotatieteen ...,Muu


['Sitten yksi sotilaista kehotti toista “lopettamaan hänet”, jonka jälkeen miestä ammuttiin päähän.',
 'Amnestyn haastatteleman todistajan mukaan mies ei kuollut heti, vaan vasta aamuyöllä.',
 'Naapurissa asuva henkilö on vahvistanut Amnestylle sotilaiden tunkeutuneen taloon ja nähneensä ammutun miehen ruumiin makaamassa pannuhuoneen nurkassa.',
 'Idässä tullaan näkemään sodan suurimmat taistelut',
 'Ruotsin maanpuolustuskorkeakoulun sotatieteen dosentin Ilmari Käihkön mukaan Venäjä on nyt muuttanut strategiaansa.']